# Neural Network 
## Activity Classification

In [ ]:
from tensorflow import keras
import polars as pl

from sklearn.preprocessing import LabelEncoder


In [ ]:
# load dataset

from lisa.config import PROCESSED_DATA_DIR
from lisa.features import sequential_stratified_split, standard_scaler


df = pl.scan_parquet(PROCESSED_DATA_DIR / "P1.parquet")

X_train, X_val, y_train, y_val = sequential_stratified_split(
    df, 0.8, 800, ["ACTIVITY"]
)

label_encoder = LabelEncoder()

X_train, X_val, scaler = standard_scaler(X_train, X_val)

X_train = X_train.to_numpy()
X_val = X_val.to_numpy()

y_train = y_train.collect().to_numpy()
y_val = y_val.collect().to_numpy()

y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

In [ ]:
# Define the model
def create_classifier(neurons1, neurons2, dropout):
    model = keras.models.Sequential([
        keras.layers.Dense(neurons1, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dropout(dropout),
        keras.layers.Dense(neurons2, activation='relu'),
        keras.layers.Dropout(dropout),
        keras.layers.Dense(3, activation='softmax')  # 3 classes for classification
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def build_classifier(hp):
    neurons1 = hp.Int("units1", min_value=32, max_value=512, step=32)
    neurons2 = hp.Int("units2", min_value=32, max_value=512, step=32)
    dropout = hp.Float("dropout", min_value=0.1, max_value=0.5, step=0.1)

    model = create_classifier(
        neurons1=neurons1, neurons2=neurons2, dropout=dropout
    )
    return model

In [ ]:
import keras_tuner as kt

tuner = kt.BayesianOptimization(
    build_classifier,
    objective='val_loss',
    max_trials=10,
    overwrite=True)

In [ ]:
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32)

In [ ]:


best_model = tuner.get_best_models()[0]
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


print(best_hps.get('units1'))
print(best_hps.get('units2'))
print(best_hps.get('dropout'))

best_model.summary()



In [ ]:
# Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# pd.DataFrame(history.history).plot(figsize=(8, 5))
# plt.grid(True)
# plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
# plt.show()

In [ ]:

test_data_path = PROCESSED_DATA_DIR / 'p15&16.parquet'

test_df = pl.read_parquet(test_data_path)

y_test = test_df.select('ACTIVITY')
X_test = test_df.select(pl.exclude(["ACTIVITY", "TRIAL", "TIME", "INCLINE", "SPEED"]))

X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

y_test = label_encoder.transform(y_test)
loss, accuracy = best_model.evaluate(X_test, y_test)

print(f'val Accuracy: {accuracy:.4f}')

## Speed Regression

In [ ]:
# Define the regression model
regression_model = keras.models.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='linear')  # Single output for regression
])

# Compile the model
regression_model.compile(
    optimizer='adam',
    loss='mse',  # Mean Squared Error for regression
    metrics=['mae']  # Mean Absolute Error as an evaluation metric
)

# Summary of the model
regression_model.summary()

In [ ]:
X_train, X_val, y_train, y_val = sequential_stratified_split(
    df, 0.8, 800, ["SPEED"]
)

label_encoder = LabelEncoder()

X_train, X_val, scaler = standard_scaler(X_train, X_val)

X_train = X_train.to_numpy()
X_val = X_val.to_numpy()

y_train = y_train.collect().to_numpy()
y_val = y_val.collect().to_numpy()

y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

In [ ]:
# Train the regression model
history = regression_model.fit(
    X_train, y_train,  # X_train: features, y_train: continuous target values
    epochs=5,
    batch_size=32,
    validation_split=0.2
)

# Evaluate the model
loss, mae = regression_model.evaluate(X_val, y_val)
print(f"Test MAE: {mae:.4f}")